In [3]:
!pip install transformers

In [2]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import json
import re

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xxl")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-xxl")

/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
tokenizer_config.json: 100%|██████████| 2.54k/2.54k [00:00<00:00, 9.12MB/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 16.0MB/s]
tokenizer.json: 100%|██████████| 2.42M/2.42M [00:00<00:00, 30.5MB/s]
special_tokens_map.json: 100%|██████████| 2.20k/2.20k [00:00<00:00, 11.0MB/s]
config.json: 100%|██████████| 674/674 [00:00<00:00, 2.41MB/s]
model.safetensors.index.json: 100%|██████████| 53.0k/53.0k [00:00<00:00, 74.2MB/s]
model-00001-of-00005.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]
model-00001-of-00005.safetensors:   0%|          | 10.5M/9.45G [00:00<01:44, 90.8MB/s]
model-00001-of-00005.safetensors:   0%|          | 21.0M/9.45G [00:00<01:42, 91.6MB/s]
model-00001-of-00005.safetensors:   0%|          | 31.5M/9.45G [00:00<

In [7]:
# Reading the test dataset back
with open('/kaggle/input/cf-dataset/test_dataset_final.json', 'r') as test_file:
    test_dataset_back = json.load(test_file)

# Display the sizes of the two datasets
#print(f"Size of the train dataset: {len(train_dataset_back)}")
print(f"Size of the test dataset: {len(test_dataset_back)}")

Size of the test dataset: 599


In [8]:
test_dataset_back[0:10]

[{'story_id': 'dd416a3f-d4ad-4314-b1c4-be55ec26ea99',
  'premise': 'Our dog ran away.',
  'initial': 'We looked everywhere for him.',
  'counterfactual': 'We looked everywhere and found him.',
  'label': 1},
 {'story_id': '73010332-6da0-4e4b-a7b2-c6d03c25a429',
  'premise': 'May really wanted to go to the concert that was playing next week.',
  'initial': 'She tried to buy tickets online, but they were sold out.',
  'counterfactual': 'May was able to buy tickets online.',
  'label': 1},
 {'story_id': '2872848e-c64d-4fcf-a741-3a1ed2aa2d00',
  'premise': 'I had a big paper due in my writing class.',
  'initial': 'I had made a mistake with my pencil and tried to erase it.',
  'counterfactual': "I had made a mistake with my pen and couldn't erase it.",
  'label': 1},
 {'story_id': '4bc3ea62-bf07-428e-b763-f438679b42af',
  'premise': 'James liked to buy scratch tickets on payday for fun.',
  'initial': 'One day he won a hundred dollars.',
  'counterfactual': 'One day he lost five dollars.',

In [9]:
# Format the TT query for the T5 model
def format_TT_query(item):
    premise = item['premise']
    initial = item['initial']
    counterfactual = item['counterfactual']
    #ask_for = item['ask-for']
    #task = "cause" if ask_for == "cause" else "effect"
    return f"I have a situation: premise: {premise}. What do you think is the most plausible reason: Initial: {initial} OR Counterfactual:{counterfactual}. Provide response in one sentence."

# Send the query to the T5 model and get the response
def query_t5(query):
    inputs = tokenizer.encode(query, return_tensors="pt")
    outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response



def normalize_text(text):
    """Normalize text by removing punctuation, converting to lowercase, and trimming spaces."""
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    return text.strip()

def is_response_correct(response, item):
    correct_hypothesis = item['initial'] if item['label'] == 0 else item['counterfactual']
    response_text = normalize_text(response)
    correct_hypothesis_normalized = normalize_text(correct_hypothesis)

    # Direct comparison for exact matches after normalization
    if response_text == correct_hypothesis_normalized:
        return True

    # Handling direct references to "hypothesis 1" or "hypothesis 2"
    if "initial" in response_text or "counterfactual" in response_text:
        hypothesis_number = "1" if "initial" in response_text else "2"
        correct_hypothesis_number = "1" if item['label'] == 0 else "2"
        return hypothesis_number == correct_hypothesis_number

    # Calculate match ratio for a more nuanced comparison
    key_phrases = correct_hypothesis_normalized.split()
    matching_phrases = [phrase for phrase in key_phrases if phrase in response_text]
    match_ratio = len(matching_phrases) / len(key_phrases)

    # Consider a response correct if a significant portion of key phrases match
    return match_ratio > 0.9  # Adjust this threshold based on experimentation and desired strictness

def evaluate_performance(data):
    correct_count = 0
    total_items = len(data)
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    skipped_items = 0

    for index, item in enumerate(data):
        query = format_TT_query(item)
        print(f"\nProcessing item {index + 1}/{total_items}...")

        response = query_t5(query)
        print(f"Generated Response: {response}")

        correct_hypothesis = item['initial'] if item['label'] == 0 else item['counterfactual']
        print(f"Correct Response: {correct_hypothesis}")

        is_correct = is_response_correct(response, item)
        print(is_correct)
        print(item['label'])
        if is_correct:
            correct_count += 1
            if item['label'] == 1:
                true_positive += 1
                print('tp')
            else:
                true_negative += 1
                print('TN')
        else:
            if item['label'] == 1:
                false_negative += 1
                print('fn')
            else:
                false_positive += 1
                print('fp')
    
    print(f"TP: {true_positive:.2f}%")
    print(f"FP: {false_positive:.2f}%")
    print(f"FN: {false_negative:.2f}%")
    print(f"total_items: {total_items}")
    print(f"CC: {correct_count}")
    #total_items -= skipped_items
    accuracy = correct_count / total_items if total_items > 0 else 0
    precision = true_positive / (true_positive + false_positive) if true_positive + false_positive > 0 else 0
    recall = true_positive / (true_positive + false_negative) if true_positive + false_negative > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    print(f"\nProcessed Items: {total_items}, Skipped Items: {skipped_items}")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision * 100:.2f}%")
    print(f"Recall: {recall * 100:.2f}%")
    print(f"F1 Score: {f1_score * 100:.2f}%")

    return accuracy, precision, recall, f1_score

In [10]:
# Evaluate performance
accuracy, precision, recall, f1_score = evaluate_performance(test_dataset_back)
print(f"Performance with T5 2 on TIMETRAVEL: Accuracy: {accuracy * 100:.3f}%, Precision: {precision * 100:.3f}%, Recall: {recall * 100:.3f}%, F1 Score: {f1_score * 100:.3f}%")



Processing item 1/599...
Generated Response: Initial: We looked everywhere for him.
Correct Response: We looked everywhere and found him.
False
1
fn

Processing item 2/599...
Generated Response: May was able to buy tickets online.
Correct Response: May was able to buy tickets online.
True
1
tp

Processing item 3/599...
Generated Response: I made a mistake with my pen and couldn't erase it.
Correct Response: I had made a mistake with my pen and couldn't erase it.
True
1
tp

Processing item 4/599...
Generated Response: One day he won a hundred dollars.
Correct Response: One day he lost five dollars.
False
1
fn

Processing item 5/599...
Generated Response: She had green dreads, glasses and beautiful blue eyes.
Correct Response: She had green dreads, glasses and beautiful blue eyes.
True
1
tp

Processing item 6/599...
Generated Response: She left anti-smoking pamphlets around the house.
Correct Response: She left anti-smoking pamphlets which convinced her mom to stop.
False
1
fn

Processi